In [ ]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler

def extract_features(file_path):
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        return None, None, None
    try:
        y, sr = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        scaler = StandardScaler()
        mfccs = scaler.fit_transform(mfccs.T).T  # Normalisasi fitur
        return mfccs, y, sr
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None, None, None

# Path file audio uji
test_audio_path = 'test.opus'

# Ekstraksi fitur dari audio uji
test_features, test_signal, sr = extract_features(test_audio_path)

# Menyimpan fitur yang diekstraksi ke dalam file teks
file_path = "test_features.txt"
np.savetxt(file_path, test_features, delimiter=',')
print(f"Extracted features saved to {file_path}")

# Visualisasi sinyal audio uji
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(test_signal, label='Audio Signal')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
plt.title('Audio Signal')
plt.legend(loc='upper right')
plt.show()
